# Q4/a

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr
import yfinance as yf
from scipy.stats import norm

yf.pdr_override()

stocks = ['AAPL', 'BP', 'COST', 'GOOG', 'GS', 'IBM', 'XOM']

endDate = "2016-12-31" 
startDate = "2016-01-01" 

# Import stock data
def getData(stocks, start, end):
    stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockData = stockData['Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return returns, meanReturns, covMatrix


returns, meanReturns, covMatrix = getData(stocks, start=startDate, end=endDate)
returns = returns.dropna()

weights = [0.15,0.15,0.15,0.20,0.05,0.20,0.10]

confidence_lvl=5

returns['portfolio'] = returns.dot(weights)



[*********************100%***********************]  7 of 7 completed


In [2]:

hist_VAR_1 = -np.percentile(returns['portfolio'], confidence_lvl)*np.sqrt(1)
hist_CVAR_1 = -returns['portfolio'][returns['portfolio'] <= np.percentile(returns['portfolio'], confidence_lvl)].mean()*np.sqrt(1)

print("1-day Hostorical VAR95% is:")
print(hist_VAR_1)
print(" ")
print("1-day Hostorical CVAR95% is:")
print(hist_CVAR_1)




1-day Hostorical VAR95% is:
0.01416876747455724
 
1-day Hostorical CVAR95% is:
0.021256080522287897


In [3]:
hist_VAR_252 = -np.percentile(returns['portfolio'], confidence_lvl)*np.sqrt(252)
hist_CVAR_252 = -returns['portfolio'][returns['portfolio'] <= np.percentile(returns['portfolio'], confidence_lvl)].mean()*np.sqrt(252)

print("252-day Hostorical VAR95% is:")
print(hist_VAR_252)
print(" ")
print("252-day Hostorical CVAR95% is:")
print(hist_CVAR_252)

252-day Hostorical VAR95% is:
0.22492221073187488
 
252-day Hostorical CVAR95% is:
0.3374298174596263


# Q4/b

In [4]:
# Portfolio Performance
def portfolioPerformance(weights, meanReturns, covMatrix, Time):
    returns = np.sum(meanReturns*weights)*Time
    std = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(Time)
    return returns, std



In [5]:
Time_1=1
pRet_1, pStd_1 = portfolioPerformance(np.array(weights), meanReturns, covMatrix, Time_1)
para_VAR_1 = -((-norm.ppf(1-confidence_lvl/100))*pStd_1 + pRet_1)
para_CVAR_1 = (confidence_lvl/100)**-1 * norm.pdf(norm.ppf(confidence_lvl/100))*pStd_1 - pRet_1


print("1-day parametric VAR95% is:")
print(para_VAR_1)
print(" ")
print("1-day parametric CVAR95% is:")
print(para_CVAR_1)






1-day parametric VAR95% is:
0.014410025028747123
 
1-day parametric CVAR95% is:
0.01820605798416901


In [6]:
Time_252=252
pRet_252, pStd_252 = portfolioPerformance(np.array(weights), meanReturns, covMatrix, Time_252)
para_VAR_252 = -((-norm.ppf(1-confidence_lvl/100))*pStd_252 + pRet_252)
para_CVAR_252 = (confidence_lvl/100)**-1 * norm.pdf(norm.ppf(confidence_lvl/100))*pStd_252 - pRet_252

print("252-day parametric VAR95% is:")
print(para_VAR_252)
print(" ")
print("252-day parametric CVAR95% is:")
print(para_CVAR_252)

252-day parametric VAR95% is:
0.10298849653931971
 
252-day parametric CVAR95% is:
0.16324865155123086


# Q4/c

Assumption: Calculated based the time range from the start date of 2016 to the end date of each month.

In [7]:
from datetime import datetime
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import calendar

stocks = ['AAPL', 'BP', 'COST', 'GOOG', 'GS', 'IBM', 'XOM']
#current_weights = np.array([0.15,0.15,0.15,0.20,0.05,0.20,0.10])


In [8]:
def sharpOptimize(price_df):
    # Calculate the expected returns and the annualized sample covariance matrix of asset returns
    mu = expected_returns.mean_historical_return(price_df)
    S = risk_models.sample_cov(price_df)
    
    # Optimize for maximum sharpe ratio
    ef = EfficientFrontier(mu,S,weight_bounds=(-0.8,0.8))
    ef.add_constraint(lambda w: w[0]+w[1]+w[2]+w[3]+w[4]+w[5]+w[6] == 1)
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights() 
    port_performance = ef.portfolio_performance(verbose=True)
    return cleaned_weights, port_performance

In [9]:
for i in range(0,12):
    start_date = dt.date(2016,1,1)
    end_of_month = dt.date(2016,i+1,calendar.monthrange(2016, i+1)[1]) 
    
    price_df = pdr.get_data_yahoo(stocks, start_date, end_of_month)['Adj Close']
    cleaned_weights, port_performance = sharpOptimize(price_df)
    print(" ")
    print("At the end of " + end_of_month.strftime('%m/%d/%Y') + ", weights should be changed to")
    print(cleaned_weights)
    print(" ")
    


[*********************100%***********************]  7 of 7 completed
Expected annual return: 136.8%
Annual volatility: 57.4%
Sharpe Ratio: 2.35
 
At the end of 01/31/2016, weights should be changed to
OrderedDict([('AAPL', -0.49623), ('BP', 0.8), ('COST', 0.69623), ('GOOG', 0.8), ('GS', -0.8), ('IBM', -0.8), ('XOM', 0.8)])
 
[*********************100%***********************]  7 of 7 completed
Expected annual return: 65.9%
Annual volatility: 33.5%
Sharpe Ratio: 1.91
 
At the end of 02/29/2016, weights should be changed to
OrderedDict([('AAPL', -0.03734), ('BP', -0.04801), ('COST', 0.11168), ('GOOG', 0.3537), ('GS', -0.8), ('IBM', 0.61997), ('XOM', 0.8)])
 
[*********************100%***********************]  7 of 7 completed
Expected annual return: 116.0%
Annual volatility: 33.1%
Sharpe Ratio: 3.45
 
At the end of 03/31/2016, weights should be changed to
OrderedDict([('AAPL', 0.47733), ('BP', -0.27577), ('COST', -0.21065), ('GOOG', 0.20909), ('GS', -0.8), ('IBM', 0.8), ('XOM', 0.8)])
 
[